# Sentiment Analysis

## Prep

### Imports

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import nltk
import pickle

### Import Stopwords

In [64]:
nltk.download('stopwords','wordnet')

[nltk_data] Downloading package stopwords to wordnet...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### import the data

In [2]:
twitdata = pd.read_csv('updated_twitter.csv', encoding = 'ISO-8859-1')
# the cscv 

In [3]:
twitdata.head()

,Unnamed: 0,target,id,date,flag,user,text,stemmed_content,lemmed_content
0,0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...,switchfoot http twitpic com zl awww bummer sho...
1,1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...,upset update facebook texting might cry result...
2,2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...,kenichan dived many time ball managed save res...
3,3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire,whole body feel itchy like fire
4,4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see,nationwideclass behaving mad see


In [8]:
twitdata['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [6]:
#here i would do some checking things like .shape() 
# and such but i already know the data is otherwis perfrct

In [7]:
twitdata.replace({'target':{4:1}}, inplace = True)

In [93]:
twitdata.to_csv('updated_twitter.csv')
twitdata

,target,id,date,flag,user,text,stemmed_content,lemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...,switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...,kenichan dived many time ball managed save res...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see,nationwideclass behaving mad see
...,...,...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,woke school best feel ever,woke school best feeling ever
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interview http b...,thewdb com cool hear old walt interview http b...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,readi mojo makeov ask detail,ready mojo makeover ask detail
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,happi th birthday boo alll time tupac amaru sh...,happy th birthday boo alll time tupac amaru sh...


### Stemming

In [11]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [20]:
twitdata['stemmed_content'] = twitdata['text'].apply(stemming)

In [21]:
twitdata.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [56]:
twitdata.stemmed_content

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object

In [57]:
twitdata.target

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

### Lemmatization

In [60]:
lemmatizer = WordNetLemmatizer()

In [66]:
import nltk 
nltk.download('wordnet')
def lemming(content):
    lemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    lemmed_content = lemmed_content.lower()
    lemmed_content = lemmed_content.split()
    lemmed_content = [lemmatizer.lemmatize(word) for word in lemmed_content if not word in stopwords.words('english')]
    lemmed_content = ' '.join(lemmed_content)
    return lemmed_content

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bolup\AppData\Roaming\nltk_data...


box run upper


In [67]:
twitdata['lemmed_content'] = twitdata['text'].apply(lemming)

# Model Training

## Stemmed

### Split data

In [28]:
X = twitdata.stemmed_content.values
Y = twitdata.target.values

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)
X_train

array(['watch saw iv drink lil wine', 'hatermagazin',
       'even though favourit drink think vodka coke wipe mind time think im gonna find new drink',
       ..., 'eager monday afternoon',
       'hope everyon mother great day wait hear guy store tomorrow',
       'love wake folger bad voic deeper'], dtype=object)

### Vectorisation

#### TfidfVectorizer

In [27]:
vectorize = TfidfVectorizer()

X_train = vectorize.fit_transform(X_train)
X_test = vectorize.transform(X_test)

##### Logistic Regression

In [28]:
model = LogisticRegression(max_iter = 1000)

In [29]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

##### Model Evaluation

In [30]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.79871953125


In [32]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy on Test Data: ', testing_data_accuracy)


Accuracy on Test Data:  0.77668125


#### Bag of Words

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
Cvectorize = CountVectorizer()
X_train = Cvectorize.fit_transform(X_train.astype('U'))
X_test = Cvectorize.transform(X_test.astype('U'))

##### Logistic Regression

In [21]:
model = LogisticRegression(max_iter = 1000)

In [22]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

###### Model Evaluation

In [23]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.82799765625


In [24]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy on Test Data: ', testing_data_accuracy)


Accuracy on Test Data:  0.775121875


#### Ngrams

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
Nvectorize = CountVectorizer(ngram_range=(1,2))
X_train = Nvectorize.fit_transform(X_train.astype('U'))
X_test = Nvectorize.transform(X_test.astype('U'))

##### Logistic Regression

In [31]:
model = LogisticRegression(max_iter = 1000)

In [32]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

###### Model Evaluation

In [33]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.958284375


In [34]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy on Test Data: ', testing_data_accuracy)


Accuracy on Test Data:  0.790028125


## Lemmatised

### Split

In [7]:
X = twitdata.lemmed_content.values
Y = twitdata.target.values

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)
X_train

array(['watch saw iv drink lil wine', 'hatermagazine',
       'even though favourite drink think vodka coke wipe mind time think im gonna find new drink',
       ..., 'eager monday afternoon',
       'hope everyone mother great day wait hear guy store tomorrow',
       'love waking folgers bad voice deeper'], dtype=object)

### Vectorising

#### TfIdf

In [70]:
vectorize = TfidfVectorizer()

X_train = vectorize.fit_transform(X_train)
X_test = vectorize.transform(X_test)

##### Logistic Regression

In [71]:
model = LogisticRegression(max_iter = 1000)

In [72]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

###### Model Evaluation

In [73]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.80987890625


In [74]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy on Test Data: ', testing_data_accuracy)


Accuracy on Test Data:  0.781903125


#### Bag of Words

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
Cvectorize = CountVectorizer()
X_train = Cvectorize.fit_transform(X_train)
X_test = Cvectorize.transform(X_test)

##### Logistic Regression

In [99]:
model = LogisticRegression(max_iter = 1000)

In [100]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

###### Model Evaluation

In [101]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.834846875


In [102]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy on Test Data: ', testing_data_accuracy)


Accuracy on Test Data:  0.780121875


#### Ngrams

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
Nvectorize = CountVectorizer(ngram_range=(1,2))
X_train = Nvectorize.fit_transform(X_train.astype('U'))
X_test = Nvectorize.transform(X_test.astype('U'))

##### Logistic Regression

In [10]:
model = LogisticRegression(max_iter = 1000)

In [11]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

###### Model Evaluation

In [12]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.963134375


In [13]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy on Test Data: ', testing_data_accuracy)


Accuracy on Test Data:  0.793534375


## Save the model & Vectorizer

In [35]:
filename = 'trained_model-stemmed-ng.sav'
pickle.dump(model,open(filename, 'wb'))
filename2 = 'vectorized-stemmed-ng.sav'
pickle.dump(Cvectorize, open(filename2, 'wb'))

### Using them

In [26]:
load_model = pickle.load(open('trained_model-stemmed-bow.sav', 'rb'))
load_vector = pickle.load(open('vectorized-stemmed-bow.sav', 'rb'))

In [ ]:
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def load_model(model_file):
    with open(model_file, 'rb') as file:
        model = pickle.load(file)
    return model

def load_vectorizer(vector_file):
    with open(vector_file, 'rb') as file:
        vectorizer = pickle.load(file)
    return vectorizer

def preprocess_text(text):
    port_stem = PorterStemmer()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [port_stem.lemmatize(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

def predict_sentiment(text, model, vectorizer):

    preprocessed_text = preprocess_text(text)
    vectorised_text = vectorizer.transform([preprocessed_text])
    sentiment = model.predict(vectorised_text)
    return sentiment[0]

def main(text):
    model = load_model("trained_model.sav")
    vectorizer = load_vectorizer("vectorized.sav")
    sentiment = predict_sentiment(text, model, vectorizer)
    if sentiment == 1:
        print("postive")
    else:
        print("negative")


main(" ")

In [80]:
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def load_model(model_file):
    with open(model_file, 'rb') as file:
        model = pickle.load(file)
    return model

def load_vectorizer(vector_file):
    with open(vector_file, 'rb') as file:
        vectorizer = pickle.load(file)
    return vectorizer

def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

def predict_sentiment(text, model, vectorizer):

    preprocessed_text = preprocess_text(text)
    vectorised_text = vectorizer.transform([preprocessed_text])
    sentiment = model.predict(vectorised_text)
    return sentiment[0]

def main(text):
    model = load_model("trained_model-Lemmed.sav")
    vectorizer = load_vectorizer("vectorized-Lemmed.sav")
    sentiment = predict_sentiment(text, model, vectorizer)
    if sentiment == 1:
        print("postive")
    else:
        print("negative")

main('I am sad')

negative


# Conclusions

## Stemming V Lemmatization
Stemming and Lemmatization have subtle differences in application, but in terms of practicality <span style="color:red">Lemmatisation</span> proved to be more resource intensive with a compute time over 2-3 times as long as <span style="color:green">Stemming</span>. but it preserved words better than <span style="color:red">stemming</span>

### Performance
Using the stemmed data, the accuracy scores for each vectorization mode were: (train, test)
> #### Stemming
Tfidf                  : 0.79871953125, 0.77668125 <br>
Bag of Words    : 0.82799765625, 0.775121875 <br>
Ngram (bigram): 0.958284375   , 0.790028125

> #### Lemmatization
Tfidf                  : 0.80987890625, 0.781903125 <br>
Bag of Words    : 0.834846875   , 0.780121875 <br>
Ngram (bigram): 0.963134375   , 0.793534375


<span style="color:green">Lemmatization</span> was consistently more accurate than <span style="color:red">Stemmed</span> data, even if only slightly and <span style="color:green">Ngram</span> representation was consistently more accurate than the other vectorizers however it produced the most overfitted models which is something to be cognisant of.

<span style="color:red">Ngram</span> also required more processing power than bothe <span style="color:green">Tfidf </span>and <span style="color:green">Bag of words</span>, without substantial improvement here. 

The Ngram model and Vectorizers were notably larger when saved, but that is neither positive nor negative when saved